In [10]:
%%capture
!pip install transformers
!pip install -U sentence-transformers
!pip install datasets
!pip install evaluate
!pip install googletrans==3.1.0a0

from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from googletrans import Translator
from sentence_transformers import SentenceTransformer, util
import sys
import os
import pandas as pd
import numpy as np



In [2]:
dataset = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/dataset_covered_recipes.csv",sep=';')#, header=0, index_col = 0)
risposta_utente = "oats,banana,milk"  #input("Hi! insert a recipe or ingredients: ") #meat
embedding_ing = emb_model.encode(risposta_utente)

EleutherAI/gpt-neo-2.7B text generation

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
tokenizer.pad_token = tokenizer.eos_token
model =  AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B", pad_token_id=tokenizer.eos_token_id)

In [11]:
if 'emb_model' not in globals():
  emb_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
dataset = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/dataset_covered_recipes.csv",sep=';')#, header=0, index_col = 0)

In [ ]:
my_ingredients = 'oats,banana,milk'
embedding_ing = emb_model.encode(my_ingredients)

In [12]:
import sys
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

dataset_sustainable =  dataset.loc[np.where(dataset['SUSTAINABILITY'].eq('HIGH'))]
dataset_sustainable.reset_index(inplace = True,drop = True)
embedding_recipes = emb_model.encode(dataset_sustainable['ingredients'])

# more sustainable
scores = []  
scores = util.pytorch_cos_sim(embedding_ing, embedding_recipes).flatten()
print(scores)
sorted = scores.argsort()[-3:]
print(scores[sorted])
high_results = dataset_sustainable.loc[sorted]

tensor([0.0759, 0.2650, 0.2650,  ..., 0.6454, 0.3984, 0.4800])
tensor([0.8304, 0.8824, 0.9423])


In [13]:
dataset_mid_sustainable =  dataset.loc[np.where(dataset['SUSTAINABILITY'].eq('MEDIUM'))]
dataset_mid_sustainable.reset_index(inplace = True,drop = True)
embedding_mid_sus_recipes = emb_model.encode(dataset_mid_sustainable['ingredients'])

# not so sustainable
mid_sus_scores = []
mid_sus_scores = util.pytorch_cos_sim(embedding_ing, embedding_mid_sus_recipes).flatten()
print(mid_sus_scores)
mid_sorted = mid_sus_scores.argsort()[-3:]
print(mid_sus_scores[mid_sorted])
medium_results = dataset_mid_sustainable.loc[sorted]

tensor([0.6187, 0.4644, 0.6124,  ..., 0.4748, 0.4394, 0.5109])
tensor([0.7735, 0.7783, 0.7811])


In [14]:
display(high_results, medium_results)

,recipe_id,title,url,ingredients,SUSTAINABILITY
487,35477,Survival Necklace,http://www.food.com/recipe/survival-necklace-9...,"banana,oat meal",HIGH
1951,50516,Strawberry Banana Smoothie,http://www.kraftrecipes.com/recipes/strawberry...,"cow milk,banana,strawberry,oat meal",HIGH
1728,2818,Oatmeal and Bananas Breakfast (Baby Food),http://www.food.com/recipe/oatmeal-and-bananas...,"oat,cow milk,banana",HIGH


,recipe_id,title,url,ingredients,SUSTAINABILITY
487,1305,Grapefruit Oil,http://cooking.nytimes.com/recipes/3950,"canola oil,grapefruit juice",MEDIUM
1951,42439,Milk Gravy (Paula Deen),http://www.food.com/recipe/milk-gravy-paula-de...,"olive oil,cow milk,wheat plain flour,black pep...",MEDIUM
1728,1100,Pressed Pie Crust,http://tastykitchen.com/recipes/desserts/press...,"olive oil,cow milk,wheat plain flour,cane sugar",MEDIUM


## High sustainable

In [4]:
prompt = ['A recipe coat,cow milk,banana']

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text[0]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'generated_text': 'A recipe cow milk,banana,strawberry,oat meal is very delicious!\n\nTried this recipe? Don’t forget to leave your feedback below!\n\nLike this'}]

In [5]:
prompt = ['What do you think about Oatmeal and Bananas Breakfast (Baby Food)? it has high sustainability']

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text[0]

[{'generated_text': 'What do you think about Strawberry Banana Smoothie? it has high sustainability index, it has rich source of nutrition & health benefits. Here are 20 healthy strawberry banana smoothie recipe below in healthy strawberry banana'}]

## Medium sustainable

In [6]:
prompt = ['A recipe cow milk,banana,strawberry,oat meal is']

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text[0]

[{'generated_text': 'A recipe cow milk,banana,strawberry,oat meal is used in making yogurt to make yogurt and many other foods.\n\nBread is made from flour. Fritters,'}]

In [7]:
prompt = ['What do you think about Fruit and Malt Oat Shake? it has medium sustainability']

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text[0]

[{'generated_text': 'What do you think about Fruit and Malt Oat Shake? it has medium sustainability and is one of our favorites of the day!\n\nA few weeks back we talked about using applesauce in'}]